In [30]:
# Import GFP dataset
import numpy as np
import pandas as pd
import csv

with open(r"C:\Users\roman\Desktop\DMS_data\DMS_data\GFP_AEQVI_Sarkisyan_2016.csv") as dms_GFP_datei:
    dms_GFP_datei_object = csv.reader(dms_GFP_datei, delimiter=',')
    print(dms_GFP_datei_object)
    for row in dms_GFP_datei_object:
        print(row)
GFP_dataset = pd.read_csv(r"C:\Users\roman\Desktop\DMS_data\DMS_data\GFP_AEQVI_Sarkisyan_2016.csv")
print(GFP_dataset)

last_character_list = []
for index, row in GFP_dataset.iterrows():
    last_character = row["mutant"][-1]
    last_character_list.append(last_character)
last_character_df = pd.DataFrame(last_character_list, columns=["new_AA"])
# Last character of each row = new amino acid

number_mutations = GFP_dataset["mutant"].str.count(":") + 1
number_mutations_Single = number_mutations == 1
# True are all the rows that only have 1 mutation

single_mutants_df = last_character_df[number_mutations_Single]
# Filters all true rows from both dataframes and creates a new one. The filtering of true values is automatically being transferred.

dms_score_df = []
for index, row in GFP_dataset.iterrows():
    dms_score = row["DMS_score"]
    dms_score_df.append(dms_score)
dms_score_df_all = pd.DataFrame(dms_score_df, columns=["DMS-score"])
# Creates a dataframe with all DMS-scores and the respective experiment number

dms_score_filtered = dms_score_df_all[number_mutations_Single]
# Dataframe with all DMS-scores of the single mutants

dms_score_filtered_new_AA = single_mutants_df.join(dms_score_filtered)
# Combines and creates a new dataframe with the new amino acid and the dms-score

mutations_pos = []
for index, row in GFP_dataset.iterrows():
    mutations_pos_number = row["mutant"][1:-1]
    mutations_pos.append(mutations_pos_number)
mutations_pos_df = pd.DataFrame(mutations_pos, columns=["Position"])
# Removes the first and last character = position

number_mutations = GFP_dataset["mutant"].str.count(":") + 1
number_mutations_Single = number_mutations == 1
# True are all the rows that only contain 1 mutation.

single_mutants_df_pos = mutations_pos_df[number_mutations_Single]
# Creates a new dataframe that has the positions of all single mutants.

mutations_pos_df_with_scores = single_mutants_df_pos.join(dms_score_filtered)
# Combines and creates a new dataframe with the position and the DMS-score (without the new amino acid)

new_column = mutations_pos_df_with_scores["Position"]
Neighbourhoods_1 = dms_score_filtered_new_AA.join(new_column)
Neighbourhoods_1 = Neighbourhoods_1[["Position", "new_AA", "DMS-score"]]
# Creates a new dataframe with position, new amino acid and DMS-score

GFP_dataset["MutationCount"] = GFP_dataset["mutant"].str.count(":") + 1
# Counts the number of mutations in each row

single_mutations_df = GFP_dataset[GFP_dataset["MutationCount"] == 1].copy()
# Filters the DataFrame to select rows with a single mutation

single_mutations_df.drop("MutationCount", axis=1, inplace=True)
# Drops the "MutationCount" column if not needed anymore

sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

AA_properties = pd.read_csv(r"C:\Users\roman\Desktop\AS Eigenschaften\aminoacids.csv")
# Import the amino acid property dataset

columns_to_delete = ['Name', 'Abbr', 'Molecular Formula', 'Residue Formula']
AA_properties.drop(columns=columns_to_delete, inplace=True)
delete_columns = ["carbon", "hydrogen", "nitrogen", "oxygen", "sulfur"]
AA_properties.drop(columns=delete_columns, inplace=True)
# Cleans the dataframe from unnecessary properties

row_index_to_delete = 12
AA_properties_to_work_with = AA_properties.drop(row_index_to_delete)
# Deletes a special amino acid that does not occur in our GFP dataset


### Generate mutated sequences and calculate properties for each mutant
import pandas as pd
import re

property_maps = {}
for property_name in AA_properties_to_work_with.columns[1:]:
    property_maps[property_name] = dict(zip(AA_properties_to_work_with["Letter"].dropna(), AA_properties_to_work_with[property_name].dropna()))
# Creates a dictionary of property maps for each property

results = []
# Creates a list to store the results

for i in range(len(sequence) - 6):
    neighbourhood = sequence[i:i + 7]  # Gets the 7-amino-acid neighbourhood sequence
    result = {"Neighbourhood": neighbourhood}
    for property_name, property_map in property_maps.items():
        score = sum(property_map.get(AA, 0) for AA in
                    neighbourhood)  # Calculates the sum of property values in the neighbourhood
        result[property_name] = score
    results.append(result)
# Calculates the neighbourhood score for every 7-amino-acid window for each property

unmutated_all_neighbourhoods = pd.DataFrame(results)
# This acts as the unmutated reference!

single_mutations_df["position"] = single_mutations_df["mutant"].apply(lambda x: int(re.search(r'\d+', x).group()))
results = []

for _, row in single_mutations_df.iterrows():
    mutation = row["mutant"]
    position = row["position"]
    mutated_sequence = unmutated_sequence[:position - 1] + mutation[-1] + unmutated_sequence[position:]

    result = {"Mutation": mutation}


    for i in range(len(mutated_sequence) - 6):
        neighbourhood = mutated_sequence[i:i + 7]
        neighbourhood_properties = {"Neighbourhood": neighbourhood}
        for property_name, property_map in property_maps.items():
            neighbourhood_properties[property_name] = sum(property_map.get(AA, 0) for AA in neighbourhood)
        results.append(neighbourhood_properties)

# Cleaning the dataframe
Neighbourhoods_2 = pd.DataFrame(results)

Neighbourhoods_2 = Neighbourhoods_2.drop_duplicates()

Neighbourhoods_2_filtered = Neighbourhoods_2.drop_duplicates()

Neighbourhoods_2_filtered = pd.concat([unmutated_all_neighbourhoods, Neighbourhoods_2_filtered], axis=0, ignore_index=True)

Neighbourhoods_3 = Neighbourhoods_2_filtered.drop_duplicates()

Neighbourhoods_4 = Neighbourhoods_3.drop(["pKx3"], axis=1)

Neighbourhoods_5 = Neighbourhoods_4.drop_duplicates()
# Final cleaned dataframe with all the single mutants and with all non-duplicated frames.
# Rows 0-231 are the unmutated reference.

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



['N149D:P192L', 'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHDVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGLVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK', '3.67758510045', '1']
['N149D:N212S', 'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHDVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPSEKRDHMVLLEFVTAAGITHGMDELYK', '3.75579065286', '1']
['N149D:N212D', 'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHDVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPDEKRDHMVLLEFVTAAGITHGMDELYK', '3.77792802799', '1']
['N149D:N198I', 'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEY

In [31]:
#Name: name of the amino acid.
#Abbr: abbreviation of the amino acid.
#Letter: letter of the amino acid.
# Molecular Weight: molecular weight.
# Molecular Formula: molecular formula.
# Residue Formula: residue formula.
# Residue Weight: residue weight (-H20)
# pKa1: the negative of the logarithm of the dissociation constant for the -COOH group.
# pKb2: the negative of the logarithm of the dissociation constant for the -NH3 group.
# pKx3: the negative of the logarithm of the dissociation constant for any other group in the molecule.
# pl4: the pH at the isoelectric point.
# H: hydrophobicity.
# VSC: volumes of side chains amino acids.
# P1: polarity.
# P2: polarizability.
# SASA: solvent accesible surface area.
# NCISC: net charge of side chains.

In [32]:
import pandas as pd

Neighbourhoods_6 = pd.DataFrame(columns=["Neighbourhood"])
columns_to_copy = ["Molecular Weight", "Residue Weight", "pKa1", "pKb2", "pl4", "H", "VSC", "P1", "P2", "SASA", "NCISC"]

unmutated_sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

for index, row in Neighbourhoods_5.iterrows():
    neighbourhood = row["Neighbourhood"]  # Gets the neighbourhood value of each row
    neighbourhood_length = len(neighbourhood)

    for i in range(
            len(unmutated_sequence) - 6):
        mismatch_found = False  # To track if a mismatch occurs

        for j in range(neighbourhood_length):
            if j == 3:  # Finds the 4th position of each neighbourhood
                if neighbourhood[j] == unmutated_sequence[
                    i + j]:  # Checks, if the 4th position matches the original sequence
                    mismatch_found = True  #If there is a match, then true
                    break  # Breaks the loop if there is a match (I dont want a match!)
            else:
                if neighbourhood[j] != unmutated_sequence[
                    i + j]:  # Checks, if the not-4th positions match the original sequence
                    mismatch_found = True  # If an amino acid matches outside of the 4th position, then true
                    break  # Breaks the loop if any other position but the 4th matches

        if not mismatch_found:  # If there was no mismatch, except for the 4th position
            Neighbourhoods_6 = pd.concat([Neighbourhoods_6, pd.DataFrame({"Neighbourhood": [neighbourhood]})],
                                ignore_index=True)  # If no mismatch, then append the neighbourhood to the new dataframe
            break  #Breaks the code because there can only be 1 match maximum

Neighbourhoods_6 = Neighbourhoods_6.join(Neighbourhoods_5[columns_to_copy])  # Adds the old columns I want to keep

,Neighbourhood,Molecular Weight,Residue Weight,pKa1,pKb2,pl4,H,VSC,P1,P2,SASA,NCISC
0,MSKSEEL,901.00,774.91,15.75,65.85,39.55,-0.98,440.9,64.7,0.990,12.126,0.269346
1,MSKDEEL,916.98,790.89,15.30,65.77,39.29,-0.43,462.3,64.2,1.059,12.320,0.304215
2,MSKAEEL,931.01,804.92,15.18,65.72,39.21,-0.30,484.3,63.6,1.105,12.547,0.302940
3,SKGKELF,859.89,733.79,15.34,66.37,35.44,1.68,384.3,61.3,0.886,11.170,0.464284
4,SKGVELF,901.97,775.87,15.83,66.16,35.43,2.01,529.8,57.7,1.295,12.952,0.323209
...,...,...,...,...,...,...,...,...,...,...,...,...
1064,GMDKLYK,783.81,657.68,15.72,66.49,35.02,0.98,370.3,62.9,0.913,11.055,0.403465
1065,GMDGLYK,807.79,681.66,15.18,66.06,36.63,-0.48,355.8,68.4,0.957,11.149,0.341103
1066,GMDDLYK,820.87,694.75,15.48,65.41,43.60,-1.08,415.8,66.7,1.071,11.820,0.382631
1067,GMDALYK,910.99,784.88,14.97,64.94,43.11,-0.37,531.3,62.9,1.361,13.167,0.241131


In [33]:
import pandas as pd

Neighbourhoods_7 = pd.DataFrame(columns=["Neighbourhood"] + columns_to_copy)

unmutated_sequence = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"

rows = []
for _, row in Neighbourhoods_6.iterrows():
    neighbourhood = row["Neighbourhood"]
    unmutated_row = neighbourhood[:3] + unmutated_sequence[3] + neighbourhood[4:]
    properties = {col: row[col] for col in columns_to_copy}
    mutated_df = pd.DataFrame(data=[[neighbourhood] + list(row[columns_to_copy])],
                              columns=["Neighbourhood"] + columns_to_copy)
    unmutated_df = pd.DataFrame(data=[[unmutated_row] + list(properties.values())],
                                columns=["Neighbourhood"] + columns_to_copy)
    rows.append(mutated_df)
    rows.append(unmutated_df)

Neighbourhoods_7 = pd.concat(rows, ignore_index=True)
# This contains an error, where he takes the 3rd position of the original sequence (absolute) and not relative to the neighbourhood how I need it

,Neighbourhood,Molecular Weight,Residue Weight,pKa1,pKb2,pl4,H,VSC,P1,P2,SASA,NCISC
0,MSKSEEL,901.00,774.91,15.75,65.85,39.55,-0.98,440.9,64.7,0.990,12.126,0.269346
1,MSKGEEL,901.00,774.91,15.75,65.85,39.55,-0.98,440.9,64.7,0.990,12.126,0.269346
2,MSKDEEL,916.98,790.89,15.30,65.77,39.29,-0.43,462.3,64.2,1.059,12.320,0.304215
3,MSKGEEL,916.98,790.89,15.30,65.77,39.29,-0.43,462.3,64.2,1.059,12.320,0.304215
4,MSKAEEL,931.01,804.92,15.18,65.72,39.21,-0.30,484.3,63.6,1.105,12.547,0.302940
...,...,...,...,...,...,...,...,...,...,...,...,...
2133,GMDGLYK,820.87,694.75,15.48,65.41,43.60,-1.08,415.8,66.7,1.071,11.820,0.382631
2134,GMDALYK,910.99,784.88,14.97,64.94,43.11,-0.37,531.3,62.9,1.361,13.167,0.241131
2135,GMDGLYK,910.99,784.88,14.97,64.94,43.11,-0.37,531.3,62.9,1.361,13.167,0.241131
2136,GMDVLYK,882.97,756.87,15.30,64.49,46.02,0.35,520.6,59.1,1.318,12.878,0.269578


In [34]:
property_names = ["Molecular Weight", "Residue Weight", "pKa1", "pKb2", "pl4", "H"]  # List of property names to calculate

Neighbourhoods_8 = Neighbourhoods_7.copy()
for property_name in property_names:
    Neighbourhoods_8[property_name] = Neighbourhoods_8["Neighbourhood"].apply(lambda x: sum(AA_properties_to_work_with[property_name].loc[AA_properties_to_work_with["Letter"] == aa].values[0] for aa in x))

# Lambda quickly defines a function without using def
# This function calculates the sum of specific property values for each letter in the "Neighbourhood" value.
# Loc finds the correct character/letter and takes the values from the columns
# Values 0 takes the first value

,Neighbourhood,Molecular Weight,Residue Weight,pKa1,pKb2,pl4,H,VSC,P1,P2,SASA,NCISC
0,MSKSEEL,931.02,804.94,15.62,65.40,39.26,-1.64,440.9,64.7,0.990,12.126,0.269346
1,MSKGEEL,901.00,774.91,15.75,65.85,39.55,-0.98,440.9,64.7,0.990,12.126,0.269346
2,MSKDEEL,959.04,832.95,15.29,65.85,36.35,-2.36,462.3,64.2,1.059,12.320,0.304215
3,MSKGEEL,901.00,774.91,15.75,65.85,39.55,-0.98,462.3,64.2,1.059,12.320,0.304215
4,MSKAEEL,915.03,788.94,15.75,65.94,39.58,-0.84,484.3,63.6,1.105,12.547,0.302940
...,...,...,...,...,...,...,...,...,...,...,...,...
2133,GMDGLYK,891.02,764.91,15.58,65.67,41.83,0.52,415.8,66.7,1.071,11.820,0.382631
2134,GMDALYK,905.05,778.94,15.58,65.76,41.86,0.66,531.3,62.9,1.361,13.167,0.241131
2135,GMDGLYK,891.02,764.91,15.58,65.67,41.83,0.52,531.3,62.9,1.361,13.167,0.241131
2136,GMDVLYK,933.10,806.99,15.56,65.69,41.82,1.12,520.6,59.1,1.318,12.878,0.269578


In [35]:
sequence = "MSKGEEL"
molecular_weight = sum(AA_properties_to_work_with["Molecular Weight"].loc[AA_properties_to_work_with["Letter"] == aa].values[0] for aa in sequence)
print("Molecular Weight:", molecular_weight)
# This is a test calculation to see if the properties of the neighbourhoods are calculated correctly.
# Comparison to manually calculating this yields the same result!

Molecular Weight: 901.0


In [36]:
Neighbourhoods_9 = Neighbourhoods_8.copy()

filtered_mutations = [mutant for mutant in single_mutations_df['mutant'] if not (len(mutant) == 3 and mutant[1] == '3')]
# Filters out mutations where 'n' in 'XnY' mutation name is 3

mutants_cloned = [mutant for mutant in filtered_mutations for _ in range(2)]
# Clones each mutant and write twice consecutively in a new list

mutants_cloned = mutants_cloned[:len(Neighbourhoods_9)]
# Truncates the list of mutants to match the length of the DataFrame

Neighbourhoods_9.insert(Neighbourhoods_9.columns.get_loc('Neighbourhood'), 'Mutation', mutants_cloned)
# Inserts the 'Mutation' column left of the 'Neighbourhood' column

     Mutation Neighbourhood  Molecular Weight  Residue Weight   pKa1   pKb2  \
0         G4S       MSKSEEL            931.02          804.94  15.62  65.40   
1         G4S       MSKGEEL            901.00          774.91  15.75  65.85   
2         G4D       MSKDEEL            959.04          832.95  15.29  65.85   
3         G4D       MSKGEEL            901.00          774.91  15.75  65.85   
4         G4A       MSKAEEL            915.03          788.94  15.75  65.94   
...       ...           ...               ...             ...    ...    ...   
2133    E235D       GMDGLYK            891.02          764.91  15.58  65.67   
2134    E235A       GMDALYK            905.05          778.94  15.58  65.76   
2135    E235A       GMDGLYK            891.02          764.91  15.58  65.67   
2136    E235V       GMDVLYK            933.10          806.99  15.56  65.69   
2137    E235V       GMDGLYK            891.02          764.91  15.58  65.67   

        pl4     H    VSC    P1     P2    SASA     N

In [37]:
Neighbourhoods_10 = Neighbourhoods_9.copy()

modified_mutations_name = []
modified_neighbourhood = []
# Creates a list to store the modified mutation names

for index, row in Neighbourhoods_9.iterrows():
    mutation = row['Mutation']
    neighbourhood = row['Neighbourhood']

    X = mutation[0]
    # Extracts the X character from the mutation name

    if index % 2 != 0:
        modified_neighbourhood.append(neighbourhood[:3] + X + neighbourhood[4:])
    else:
        modified_neighbourhood.append(neighbourhood)
   # Modifies the 4th position of the neighbourhood sequence based on X for odd rows

    if index % 2 != 0:
        modified_mutations_name.append(mutation + '-unmut')
    else:
        modified_mutations_name.append(mutation)
  # Appends '-unmut' to every other mutation name

Neighbourhoods_10['Mutation'] = modified_mutations_name
Neighbourhoods_10['Neighbourhood'] = modified_neighbourhood
# Updates the 'Mutation' and 'Neighbourhood' columns with the modified values

         Mutation Neighbourhood  Molecular Weight  Residue Weight   pKa1  \
0             G4S       MSKSEEL            931.02          804.94  15.62   
1       G4S-unmut       MSKGEEL            901.00          774.91  15.75   
2             G4D       MSKDEEL            959.04          832.95  15.29   
3       G4D-unmut       MSKGEEL            901.00          774.91  15.75   
4             G4A       MSKAEEL            915.03          788.94  15.75   
...           ...           ...               ...             ...    ...   
2133  E235D-unmut       GMDELYK            891.02          764.91  15.58   
2134        E235A       GMDALYK            905.05          778.94  15.58   
2135  E235A-unmut       GMDELYK            891.02          764.91  15.58   
2136        E235V       GMDVLYK            933.10          806.99  15.56   
2137  E235V-unmut       GMDELYK            891.02          764.91  15.58   

       pKb2    pl4     H    VSC    P1     P2    SASA     NCISC  
0     65.40  39.26 -1.

In [39]:
property_names = ["Molecular Weight", "Residue Weight", "pKa1", "pKb2", "pl4", "H", "VSC", "P1", "P2", "SASA",
                  "NCISC"]  # List of property names to calculate

Neighbourhoods_11 = Neighbourhoods_10.copy()
for property_name in property_names:
    Neighbourhoods_11[property_name] = Neighbourhoods_11["Neighbourhood"].apply(
        lambda x: sum(AA_properties_to_work_with[property_name].loc[AA_properties_to_work_with["Letter"] == aa].values[0] for aa in x))

# This calculates all values for the newly inserted unmutated sequences and fills them in (to update the values)
# The values seem to be correct according to manual checking!
# All the mutations bevor position 4 and after 235 cannot be seen in this dataframe, because they can never be in the middle of a neighbourhood of length 7

         Mutation Neighbourhood  Molecular Weight  Residue Weight   pKa1  \
0             G4S       MSKSEEL            931.02          804.94  15.62   
1       G4S-unmut       MSKGEEL            901.00          774.91  15.75   
2             G4D       MSKDEEL            959.04          832.95  15.29   
3       G4D-unmut       MSKGEEL            901.00          774.91  15.75   
4             G4A       MSKAEEL            915.03          788.94  15.75   
...           ...           ...               ...             ...    ...   
2133  E235D-unmut       GMDELYK            963.08          836.98  15.43   
2134        E235A       GMDALYK            905.05          778.94  15.58   
2135  E235A-unmut       GMDELYK            963.08          836.98  15.43   
2136        E235V       GMDVLYK            933.10          806.99  15.56   
2137  E235V-unmut       GMDELYK            963.08          836.98  15.43   

       pKb2    pl4     H    VSC    P1     P2    SASA     NCISC  
0     65.40  39.26 -1.

In [40]:
# Enter a specific mutation in the little window that pops up!

mutation_name = input("Enter the mutation name: ")
Dataframe_for_specific_mutation = Neighbourhoods_11[Neighbourhoods_11['Mutation'].str.contains(mutation_name)]
print(Dataframe_for_specific_mutation)
# works!

         Mutation Neighbourhood  Molecular Weight  Residue Weight   pKa1  \
1446        V163A       GIKANFK            885.04          758.94  15.25   
1447  V163A-unmut       GIKVNFK            913.09          786.99  15.23   

       pKb2    pl4     H    VSC    P1     P2    SASA     NCISC  
1446  64.72  48.36 -0.11  495.2  61.7  1.094  12.271  0.286230  
1447  64.65  48.32  0.35  539.2  59.5  1.188  12.735  0.336047  
